In [2]:
branca = 0.3931/0.0334
print(branca)

11.769461077844312


In [3]:
import math

# Número que será o expoente
expoente = branca

# Calculando e^expoente
resultado = math.exp(branca)

print(f"e^{expoente} = {resultado}")
# Saída: e^3 ≈ 20.085536923187668

e^11.769461077844312 = 129244.47926537614


In [4]:
import numpy as np
import pandas as pd

# Coefficients from Model 5 (most complete model)
coef = {
    'AMARELA': 0.7017,
    'BRANCA': 0.3931,  # White advantage over Black (reference)
    'FEMININO': -0.8912,
    'FUNDO_ESPECIAL': 0.0334,
    'PARDA': 0.3000,
    'INDIGENA': 0.0817,
    'Constante': -3.7351,
    # Other controls (kept constant in comparison)
    'ENSINO_FUNDAMENTAL_COMPLETO': -0.6243,
    'GOVERNADOR': 0.9913,
    'LOG_EDUCACAO': -0.1503,
    'LOG_SAUDE_ASSIST_SOCIAL': 0.1387,
    'PRESIDENTE': 0.9931,
    'REELEICAO': 2.3309
}

def calcular_fundo_equivalente(grupo_alvo, grupo_referencia, coeficientes):
    """
    Calcula quanto financiamento adicional um grupo-alvo precisa para igualar 
    a probabilidade de eleição do grupo de referência.
    
    Parâmetros:
    grupo_alvo: tupla - (raça, gênero) do grupo sendo analisado
    grupo_referencia: tupla - (raça, gênero) do grupo de referência
    coeficientes: dict - Coeficientes do modelo
    
    Retorna:
    (diferenca_log, fator) - (diferença em log necessária, fator multiplicativo)
    """
    def efeito_grupo(raca, genero):
        coef_raca = coeficientes.get(raca, 0)  # 0 para PRETA
        coef_genero = coeficientes['FEMININO'] if genero == 'FEMININO' else 0
        return coef_raca + coef_genero
    
    efeito_alvo = efeito_grupo(*grupo_alvo)
    efeito_referencia = efeito_grupo(*grupo_referencia)
    diferenca_efeito = efeito_referencia - efeito_alvo
    
    # Coeficiente do fundo de campanha
    beta_fundo = coeficientes['FUNDO_ESPECIAL']
    
    if beta_fundo == 0:
        return (np.inf, np.inf)
    
    diferenca_log = diferenca_efeito / beta_fundo
    fator = np.exp(diferenca_log)
    
    return (diferenca_log, fator)

# Grupo de referência (Homem branco)
REFERENCIA = ('BRANCA', 'MASCULINO')

# Grupos para comparação
grupos = [
    ('PRETA', 'MASCULINO'),
    ('PRETA', 'FEMININO'),
    ('BRANCA', 'FEMININO'),  # Mulheres brancas
    ('PARDA', 'MASCULINO'),
    ('PARDA', 'FEMININO'),
    ('AMARELA', 'MASCULINO'),
    ('AMARELA', 'FEMININO'),
    ('INDIGENA', 'MASCULINO'),
    ('INDIGENA', 'FEMININO')
]

# Calcular resultados
resultados = []
for grupo in grupos:
    diferenca_log, fator = calcular_fundo_equivalente(grupo, REFERENCIA, coef)
    resultados.append({
        'Raça': grupo[0],
        'Gênero': grupo[1],
        'Δ log(Financiamento)': diferenca_log,
        'Fator Multiplicativo': fator,
        'Interpretação': f"Igual" if fator == 1 else f"{fator:.1f}x financiamento"
    })

# Criar tabela de comparação
tabela_resultados = pd.DataFrame(resultados)
print("Financiamento Necessário para Igualar Probabilidade de Eleição de Homens Brancos:")
print(tabela_resultados.to_string(float_format="%.1f", index=False))

# Caso especial: Candidatas mulheres negras
diferenca_mulher_negra, fator_mulher_negra = calcular_fundo_equivalente(
    ('PRETA', 'FEMININO'), REFERENCIA, coef)
print(f"\nCandidatas mulheres negras precisam de {fator_mulher_negra:,.0f} vezes ")
print(f"o financiamento de homens brancos para ter probabilidade igual de eleição")

# Caso mulheres brancas
diferenca_mulher_branca, fator_mulher_branca = calcular_fundo_equivalente(
    ('BRANCA', 'FEMININO'), REFERENCIA, coef)
print(f"Candidatas mulheres brancas precisam de {fator_mulher_branca:,.0f} vezes ")
print(f"o financiamento de homens brancos")

# Mostrar equação de probabilidade
print("\nEquação de probabilidade para candidato homem branco:")
print(f"P = 1 / (1 + e^(-(β0 + β_BRANCA + outros_controles)))")

Financiamento Necessário para Igualar Probabilidade de Eleição de Homens Brancos:
    Raça    Gênero  Δ log(Financiamento)  Fator Multiplicativo                      Interpretação
   PRETA MASCULINO                  11.8              129244.5            129244.5x financiamento
   PRETA  FEMININO                  38.5   50064862437228456.0 50064862437228456.0x financiamento
  BRANCA  FEMININO                  26.7        387365578180.2      387365578180.2x financiamento
   PARDA MASCULINO                   2.8                  16.2                16.2x financiamento
   PARDA  FEMININO                  29.5       6290488711720.0     6290488711720.0x financiamento
 AMARELA MASCULINO                  -9.2                   0.0                 0.0x financiamento
 AMARELA  FEMININO                  17.4            37622535.3          37622535.3x financiamento
INDIGENA MASCULINO                   9.3               11196.5             11196.5x financiamento
INDIGENA  FEMININO                  